In [2]:
import numpy as np
import qutip as qt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.circuit.library import QFT
from random import randint
import matplotlib.pyplot as plt
from time import time


from tools.classical import *
from tools.quantum import *
from boltzmann import *
from op_fourier_trafo import *

In [7]:

num_energy_bits = 6
num_qubits = 3
qr_boltzmann = QuantumRegister(1, name='qr_boltz')
qr_energy = QuantumRegister(num_energy_bits, name='qr_energy')

cr_boltzmann = ClassicalRegister(1, name='cr_boltz')
cr_energy = ClassicalRegister(num_energy_bits, name='cr_energy')

bithandler = BitHandler([cr_boltzmann, cr_energy])

circ0 = QuantumCircuit(qr_boltzmann, qr_energy, cr_boltzmann, cr_energy)
circ1 = QuantumCircuit(qr_boltzmann, qr_energy, cr_boltzmann, cr_energy)

boltz_circ = lookup_table_boltzmann(num_energy_bits)

#* By hand
t0 = time()
inverse_boltz_circ = reverse_lookup_table_boltzmann(num_energy_bits)
circ0.compose(boltz_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)
circ0.compose(inverse_boltz_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)
tr_circ0 = transpile(circ0, basis_gates=['cx', 'u3'], optimization_level=3)
t1 = time()
#* By Qiskit
t2 = time()
qiskit_inverse_boltz_circ = boltz_circ.inverse()
circ1.compose(boltz_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)
circ1.compose(qiskit_inverse_boltz_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)
tr_circ1 = transpile(circ1, basis_gates=['cx', 'u3'], optimization_level=3)
t3 = time()

op0 = Operator(circ0)
op1 = Operator(circ1)
dist_to_identity0 = np.linalg.norm(op0.data - np.eye(2**(1+num_energy_bits)))
dist_to_identity1 = np.linalg.norm(op1.data - np.eye(2**(1+num_energy_bits)))

print(f"By hand: {t1-t0:.3f} s")
print(dist_to_identity0)
print(f"By Qiskit: {t3-t2:.3f} s")
print(dist_to_identity1)

/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


By hand: 6.157 s
5.609877175673462e-13
By Qiskit: 7.610 s
5.609877175673462e-13


In [3]:
np.random.seed(667)
num_qubits = 3
num_energy_bits = 6
bohr_bound = 2 ** (-num_energy_bits + 1) #!
eps = 0.1
sigma = 10
eig_index = 0
T = 1
shots = 100

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)
rescaled_coeff = hamiltonian.rescaled_coeffs
# Corresponding Trotter step circuit
trotter_step_circ = trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, symbreak=True)
hamiltonian.trotter_step_circ = trotter_step_circ

Original spectrum:  [-3.4    -2.8921 -2.2725 -1.4     1.3262  2.1779  2.9659  3.4946]
Ideal spectrum:  [0.0233 0.0572 0.0985 0.1567 0.3384 0.3952 0.4477 0.483 ]
Nonrescaled coefficients:  [0.9 0.7 0.8 1. ]
Rescaled coefficients:  [0.06       0.04666667 0.05333333 0.06666667]


In [8]:
qr_energy = QuantumRegister(num_energy_bits, name='qr_energy')
qr_sys = QuantumRegister(num_qubits, name='qr_sys')
cr_energy = ClassicalRegister(num_energy_bits, name='cr_energy')
cr_sys = ClassicalRegister(num_qubits, name='cr_sys')

bithandler = BitHandler([cr_energy, cr_sys])

circ0 = QuantumCircuit(qr_energy, qr_sys, cr_energy, cr_sys)
circ1 = QuantumCircuit(qr_energy, qr_sys, cr_energy, cr_sys)

#* Ini
# if sigma != 0.:
#     prep_circ = brute_prepare_gaussian_state(num_energy_bits, sigma)
#     circ.compose(prep_circ, [*list(qr_energy)], inplace=True)
# else:  # Conventional QPE
#     circ.h(qr_energy)
    
# System prep, initial state = eigenstate
initial_state = np.array([1] + [0] * (2**num_qubits - 1))

#* OFT
jump_op = Operator(Pauli('X'))
oft_circ = operator_fourier_circuit(jump_op, num_qubits, num_energy_bits, hamiltonian, 
                                    initial_state=initial_state, sigma=sigma)
# oft_op = Operator(oft_circ)
#* By hand
t0 = time()
inverse_oft_circ = inverse_operator_fourier_transform(jump_op, num_qubits, num_energy_bits, hamiltonian, sigma=sigma)
# inverse_oft_op = Operator(inverse_oft_circ)
circ0.compose(oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)
circ0.compose(inverse_oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)
tr_circ0 = transpile(circ0, basis_gates=['cx', 'u3'], optimization_level=3)
t1 = time()

#* By Qiskit
# t2 = time()
# qiskit_inverse_oft_circ = oft_circ.inverse()
# circ1.compose(oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)
# circ1.compose(qiskit_inverse_oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)
# # tr_circ1 = transpile(circ1, basis_gates=['cx', 'u3'], optimization_level=3)
# t3 = time()

# op0 = Operator(circ0)
# op1 = Operator(circ1)
# dist_to_identity0 = np.linalg.norm(op0.data - np.eye(2**(1+num_energy_bits)))
# dist_to_identity1 = np.linalg.norm(op1.data - np.eye(2**(1+num_energy_bits)))

# print(f"By hand: {t1-t0:.3f} s")
# print(dist_to_identity0)
print(f"By Qiskit: {t3-t2:.3f} s")
# print(dist_to_identity1)

Energy before jump: 0.4766666666666667
Jump applied to 1th qubit
Energy after jump: 0.13
Energy jump: -0.3466666666666667
Jump applied to 1th qubit


/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


KeyboardInterrupt: 